In [1]:
from selenium import webdriver
import selenium
import time
import pymongo
from bs4 import BeautifulSoup
from urllib.parse import (urljoin, urlparse)
import requests

In [12]:
def go_to_player_page(player_name):
    
    #create selenium instance
    browser = webdriver.Chrome()
    
    #go to player page on ChessTempo
    url = 'https://chesstempo.com/chess-players.html'
    browser.get(url)

    #Input player name to submition box
    search_box = browser.find_element_by_css_selector("input#playerName")
    search_box.send_keys(player_name)

    #click search button
    search_button = browser.find_element_by_css_selector('#submitbutton')
    search_button.click()
    time.sleep(1)

    #click on first player entry from search results
    player = browser.find_element_by_css_selector('#board-playerList td.yui-dt-col-surname div.yui-dt-liner a')
    player.click()
    time.sleep(1)

    #click number of rows selector and select 500 rows
    browser.find_element_by_xpath("//select[@title='Rows per page']/option[text()='500']").click()
    time.sleep(2)
    
    #return html source
    return browser.page_source


In [3]:
# morphy_html = go_to_player_page('Morphy, Paul')

In [21]:
def pgn_links_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select('a.user-link')
    url = 'https://chesstempo.com/'
    
    #get relative links from beatutiful soup selector
    pgn_links_rel = [link.attrs.get('href') for link in links
                     if 'download_game_pgn' in link.attrs.get('href')]
    
    #join the relative links to the base url for ChessTempo and return pgn_links
    return [urljoin(url, link) for link in pgn_links_rel]

In [5]:
pgn_links

NameError: name 'pgn_links' is not defined

In [14]:
def scrape_pgn(url):
    #grab fake credentials from selenium session
    ##Under Referer player number doesn't change which might be an issue 
    raw_headers = '''Host: chesstempo.com
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
Referer: https://chesstempo.com/gamedb/player/243342
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US,en;q=0.9
Cookie: PHPSESSID=slood7ckip4i7qg4271qgtsfm1; _ga=GA1.2.1156113456.1525794011; _gid=GA1.2.1594640290.1525794011'''

    #put headers into a dict so it can work with requests
    headers = dict([h.split(': ') for h in raw_headers.split('\n')])
    
    #request the pgn files using fake credentials
    response = requests.get(url, headers=headers)
    
    #check if the request worked
    assert response.status_code == 200 
    
    #grab pgn text
    return response.content

In [15]:
def scrape_pgn_links(pgn_links):
    for link in pgn_links:
        game_id = link.split('gameids=')[1]
        yield (game_id, scrape_pgn(link))
        time.sleep(4)

In [16]:
def save_pgn_file(game_id, data):
    file_name = '/Users/bmcgarry/galvanize/ChessCapstone/ChessStyle/data/chessTempoPGN/' + game_id + '.pgn'
    with open(file_name, 'wb') as f:
        f.write(data)
    

In [17]:
def get_pgn_files(pgn_links):
    for (game_id, data) in scrape_pgn_links(pgn_links):
        save_pgn_file(game_id, data)

In [ ]:
get_pgn_files(pgn_links[4:7])

In [18]:
def get_player_pgn_files(player_name):
    html = go_to_player_page(player_name)
    pgn_links = pgn_links_from_html(html)
    get_pgn_files(pgn_links[:10])

In [22]:
get_player_pgn_files('Morphy, Paul')